In [1]:
!pip install av datasets
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 46.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-c

In [2]:
import json
import multiprocessing
import os

import av
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer, AutoImageProcessor

2025-04-19 19:42:05.506724: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745091725.766432      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745091725.839769      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Login to Huggingface

In [5]:
from huggingface_hub import notebook_login

notebook_login()  # will prompt you for your HF token

## Load Data

In [3]:
# prompt: convert csv to df

import pandas as pd

# Assuming your CSV file is named 'your_file.csv' and is in the current directory.
# Replace 'your_file.csv' with the actual path to your CSV file if it's located elsewhere.
df = pd.read_csv('/kaggle/input/splitted-ucf-120videospercategory/splitted-ucf-clips.csv')
# Remove "./splitted_clips/" from the 'video_path' column
df['video_path'] = df['video_path'].str.replace('./splitted_clips/', '')
# Display the first few rows of the DataFrame to verify
df

,video_path,caption
0,Normal_Videos615_x264_clip0.mp4,"The camera locks on a building, and then the c..."
1,Normal_Videos615_x264_clip1.mp4,The bottom of the lens moves downward and then...
2,Normal_Videos615_x264_clip2.mp4,"The camera moves to the right, shines on the p..."
3,Normal_Videos615_x264_clip3.mp4,Then the camera moves to the left to illuminat...
4,Normal_Videos615_x264_clip4.mp4,The camera moves to the right to the highway. ...
...,...,...
9645,Vandalism025_x264_clip11.mp4,The man in blue slowly walked to the gray obje...
9646,Vandalism047_x264_clip0.mp4,A man in black clothes and a blue hat punched ...
9647,Vandalism047_x264_clip1.mp4,The man in blue hat and black clothes continue...
9648,Vandalism047_x264_clip2.mp4,The man in blue hat and black clothes turned a...


In [6]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from datasets import Dataset

# Assuming 'df' is your DataFrame and it includes a 'video_path' column
df['video_group'] = df['video_path'].str.split('_').str[0]  # Adjust based on actual naming

# Step 1: Train-Test split (85% train+val, 15% test)
gss1 = GroupShuffleSplit(test_size=0.05, n_splits=1, random_state=42)
train_val_inds, test_inds = next(gss1.split(df, groups=df['video_group']))
train_val_df = df.iloc[train_val_inds].reset_index(drop=True)
test_df = df.iloc[test_inds].reset_index(drop=True)

# Step 2: Train-Validation split (from 85% of original data → 70% train, 15% val overall)
gss2 = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=42)  
# 0.1765 ≈ 15 / 85 → ensures ~15% of full data goes to validation
train_inds, val_inds = next(gss2.split(train_val_df, groups=train_val_df['video_group']))
train_df = train_val_df.iloc[train_inds].reset_index(drop=True)
valid_df = train_val_df.iloc[val_inds].reset_index(drop=True)

# Shuffle for randomness
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
valid_df = valid_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert to Hugging Face Datasets format
dataset = {
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(valid_df[:500]),
    "test": Dataset.from_pandas(test_df[:100])
}

# Display sizes
print(f"Train: {len(train_df)}, Validation: {len(valid_df)}, Test: {len(test_df)}")

Train: 8397, Validation: 716, Test: 537


Load Captions

In [7]:
# dataset = load_dataset("json", data_files={"train": "../dataset/vatex_train.json", "validation": "../dataset/vatex_val.json"}, num_proc=os.cpu_count()).remove_columns("chCap")
# dataset["test"] = load_dataset("json", data_files={"test": "../dataset/vatex_test.json"}, num_proc=os.cpu_count())["test"]
# dataset

Save {"videoID": [captions]} Map

In [8]:
# def make_jsons(example, videoID_captions=None):
#     videoID, captions = example["videoID"], example["enCap"]
#     videoID_captions[videoID] = captions
#     return example

# manager = multiprocessing.Manager()
# videoID_captions = manager.dict()
# dataset.map(make_jsons, fn_kwargs={"videoID_captions": videoID_captions}, num_proc=os.cpu_count())

# with open("../dataset/videoID_captions.json", "w") as file:
#     file.write(json.dumps(videoID_captions.copy()))

Tokenize Data and Save as HF Dataset

In [7]:
from datasets import DatasetDict
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# create dataset with features: videoID, pixel_values (8, 3, 224, 224), labels (10, 1024)
def process(example):
    videoID, captions = example["video_path"], example["caption"]
    
    videos_path = "/kaggle/input/splitted-ucf-120videospercategory/splitted-ucf-clips"
    video_path = os.path.join(videos_path, videoID)
    if not os.path.isfile(video_path):
        video_path = os.path.join(videos_path, videoID)
    container = av.open(video_path)
    
    # discrepancy between in codec metadata, manually get frame count
    container.seek(0)
    frame_count = 0
    for frame in container.decode(video=0):
        frame_count += 1
    
    indices = set(np.linspace(0, frame_count, num=8, endpoint=False).astype(np.int64))
    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
            frames.append(frame.to_ndarray(format="rgb24"))   
            
    pixel_values = image_processor(frames).pixel_values[0]
    labels = tokenizer(captions, padding="max_length").input_ids
    return {"videoID": videoID, "pixel_values": pixel_values, "labels": labels}

dataset["train"] = dataset["train"].map(
    process,
    remove_columns=["caption"],
    num_proc=1,
    cache_file_name="/kaggle/tmp/dataset-ucf/cache",    # ← write here
    writer_batch_size=500                          # flush every 500 examples
)
dataset["validation"] = dataset["validation"].map(process, remove_columns=["caption"], num_proc=1)
dataset["test"] = dataset["test"].map(process, remove_columns=["caption"], num_proc=1)


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/8397 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [16]:
from datasets import DatasetDict

# Wrap your dict into a DatasetDict
dataset_dict = DatasetDict({
    "train": dataset["train"],
    "validation": dataset["validation"],
    "test": dataset["test"]
    
})

In [17]:
from huggingface_hub import HfApi


api = HfApi()

# Create the repo (it will be public by default unless private=True)
api.create_repo(
    repo_id="NourFakih/ucf101-captioned-mapped",
    repo_type="dataset",
    exist_ok=True  # Avoid error if it already exists
)



# make your existing dataset public
api.update_repo_visibility(
    repo_id="NourFakih/ucf101-captioned-mapped",
    repo_type="dataset",
    private=False
)  # :contentReference[oaicite:0]{index=0}
# Push to Hugging Face Hub

dataset_dict.push_to_hub("NourFakih/ucf101-captioned-mapped", private=False)

Uploading the dataset shards:   0%|          | 0/82 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/623 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/NourFakih/ucf101-captioned-mapped/commit/fa2abf76bcbfa86c6bf9f84224d25ed00f0c477f', commit_message='Upload dataset (part 00001-of-00002)', commit_description='', oid='fa2abf76bcbfa86c6bf9f84224d25ed00f0c477f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/NourFakih/ucf101-captioned-mapped', endpoint='https://huggingface.co', repo_type='dataset', repo_id='NourFakih/ucf101-captioned-mapped'), pr_revision=None, pr_num=None)

In [22]:
# import shutil

# shutil.rmtree("/kaggle/tmp/dataset-ucf/cache", ignore_errors=True)
# import shutil
# shutil.rmtree("/kaggle/tmp/", ignore_errors=True)



In [14]:
# import shutil
# shutil.rmtree("/kaggle/working/dataset-ucf", ignore_errors=True)

In [15]:
# from datasets import DatasetDict
# dataset = DatasetDict(dataset)
# #dataset.save_to_disk("./dataset-ucf", num_proc=1)
# # Save locally first
# dataset.save_to_disk("/kaggle/tmp/dataset-ucf")



Saving the dataset (0/68 shards):   0%|          | 0/7000 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [23]:
!df -h

Filesystem                                                              Size  Used Avail Use% Mounted on
overlay                                                                 7.9T  6.2T  1.8T  79% /
tmpfs                                                                    64M     0   64M   0% /dev
shm                                                                      14G  4.0K   14G   1% /dev/shm
/dev/loop1                                                               20G   76K   20G   1% /kaggle/lib
192.168.3.2:/data/kagglesdsdata/datasets/7105581/11354553/d9837axsw7ca   78T   59T   20T  75% /kaggle/input/splitted-clips-ucf
192.168.3.2:/data/kagglesdsdata/datasets/7154027/11423193/d9837axuj6py   78T   59T   20T  75% /kaggle/input/splitted-ucf-120videospercategory
/dev/mapper/snap                                                        7.9T  6.2T  1.8T  79% /etc/hosts
tmpfs                                                                    16G     0   16G   0% /proc/acpi
tmpfs       

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
# import gc
# gc.collect()


16328

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'update_repo_visibility' (from 'huggingface_hub.hf_api') is deprecated and will be removed from version '0.32'. Please use `update_repo_settings` instead.
  warnings.warn(warning_message, FutureWarning)


{'private': False}

Uploading the dataset shards:   0%|          | 0/68 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/NourFakih/ucf101-captioned/commit/60d181d80880bbb9bb8fd51b0bb55254be407ccc', commit_message='Upload dataset (part 00001-of-00002)', commit_description='', oid='60d181d80880bbb9bb8fd51b0bb55254be407ccc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/NourFakih/ucf101-captioned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='NourFakih/ucf101-captioned'), pr_revision=None, pr_num=None)

In [ ]:
# !zip -r /kaggle/working/ucf_dataset.zip /kaggle/tmp/dataset-ucf



In [ ]:
# !pip install kaggle

In [ ]:
# import os
# import json

# # Create .kaggle directory
# os.makedirs("/root/.kaggle", exist_ok=True)

# # Move kaggle.json to correct location
# !cp /kaggle/working/kaggle.json /root/.kaggle/kaggle.json

# # Set permissions
# !chmod 600 /root/.kaggle/kaggle.json


In [ ]:
# dataset_metadata = {
#     "title": "UCF101 Video Captioning Mapped Dataset",
#     "id": "nourfakih/ucf101-captioned-mapped",
#     "licenses": [{"name": "CC0-1.0"}]
# }
# with open("/kaggle/working/dataset-metadata.json", "w") as f:
#     json.dump(dataset_metadata, f)


In [ ]:
# !kaggle datasets create \
#   -p /kaggle/working \
#   -u \
#   -r zip \
#   --dir-mode zip


In [ ]:
# from datasets import DatasetDict
# dataset = DatasetDict(dataset)
# dataset.save_to_disk("./dataset-ucf", num_proc=1)

In [ ]:
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()


In [ ]:
# from datasets import load_dataset

# # 1. Load all splits at once
# dataset = load_dataset("NourFakih/ucf101-captioned")  
# # `dataset` is a DatasetDict with keys like "train", "validation", "test" :contentReference[oaicite:0]{index=0}

# train_ds      = dataset["train"]
# validation_ds = dataset["validation"]
# test_ds       = dataset["test"]

README.md:   0%|          | 0.00/722 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/68 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/68 [00:00<?, ?it/s]

train-00000-of-00068.parquet:   0%|          | 0.00/119M [00:00<?, ?B/s]

train-00001-of-00068.parquet:   0%|          | 0.00/130M [00:00<?, ?B/s]

...

validation-00003-of-00005.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

validation-00004-of-00005.parquet:   0%|          | 0.00/123M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/122M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

## Load Base model

In [ ]:
# import json
# import os

# from datasets import Dataset, load_from_disk
# import numpy as np
# import torch
# from torch.cuda.amp import GradScaler
# from torch.optim import AdamW
# from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter
# from tqdm.auto import tqdm
# from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, default_data_collator, get_scheduler

# device = "cuda"

# # MODEL
# encoder = "facebook/timesformer-base-finetuned-k600"
# decoder = "gpt2"

# image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
# tokenizer = AutoTokenizer.from_pretrained(decoder)
# tokenizer.pad_token = tokenizer.eos_token

# model_name="Neleac/SpaceTimeGPT"
# model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)
# #model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)
# model.config.decoder_start_token_id = tokenizer.bos_token_id
# model.config.pad_token_id = tokenizer.pad_token_id
# model.config.max_length = 50
# model.config.num_beams = 4
# model.config.early_stopping = True


## Prepare Data for training

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # Disable tokenizers parallelism to avoid deadlocks
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # Imports
# import torch
# from torch.utils.data import Dataset, DataLoader
# from transformers import VisionEncoderDecoderModel , get_scheduler
# from transformers import default_data_collator
# from torch.amp import autocast, GradScaler
# from datasets import load_from_disk

# # ───────────────────────────────────────────────────────────────────────────────
# # 1. Dataset definition and loading
# class UCFDataset(Dataset):
#     def __init__(self, dataset):
#         self.dataset = dataset

#     def __len__(self):
#         # expand each video to 10 captions
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         video_idx   = idx
#         caption_idx = idx
#         example     = self.dataset[video_idx]
#         return {
#             "video_path":   example["video_path"],
#             "pixel_values": example["pixel_values"],
#             "labels":       example["labels"][caption_idx]
#         }

# # load from disk and set PyTorch format
# dataset = load_from_disk("./dataset-ucf")
# dataset.set_format("torch")
# dataset_train = UCFDataset(dataset["train"])
# dataset_val   = UCFDataset(dataset["validation"])

# # ───────────────────────────────────────────────────────────────────────────────
# # 2. Collators to ensure correct tensor shapes
# def train_collator(examples):
#     pixel_values = torch.stack([ex["pixel_values"] for ex in examples])
#     labels       = torch.stack([ex["labels"]       for ex in examples])
#     return {"pixel_values": pixel_values, "labels": labels}

# def val_collator(examples):
#     pixel_values = torch.stack([ex["pixel_values"] for ex in examples])
#     labels       = torch.stack([ex["labels"]       for ex in examples])
#     video_paths  = [ex["video_path"] for ex in examples]
#     return {"pixel_values": pixel_values, "labels": labels, "video_paths": video_paths}

# # ───────────────────────────────────────────────────────────────────────────────
# # 3. DataLoaders
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train_dataloader = DataLoader(
#     dataset_train,
#     collate_fn = train_collator,
#     batch_size  = 1,
#     drop_last   = True,
#     num_workers = 1,
#     pin_memory  = True,
# )
# val_dataloader = DataLoader(
#     dataset_val,
#     collate_fn = val_collator,
#     batch_size  = 1,
#     drop_last   = False,
#     num_workers = 1,
#     pin_memory  = True,
# )


In [ ]:

# # ───────────────────────────────────────────────────────────────────────────────
# # 4. Model, optimizer, scheduler, and AMP scaler
# model     = VisionEncoderDecoderModel.from_pretrained("Neleac/SpaceTimeGPT").to(device)
# optimizer = AdamW(model.parameters(), lr=5e-7)
# epochs    = 1
# total_steps = epochs * len(train_dataloader)
# scheduler   = get_scheduler(
#     name             = "linear",
#     optimizer        = optimizer,
#     num_warmup_steps = 0,
#     num_training_steps = total_steps,
# )
# scaler = GradScaler("cuda")  # :contentReference[oaicite:0]{index=0}

####  Video Captioning Training

In [ ]:
# from tqdm.auto import tqdm  # auto handles Jupyter vs script

# # Training loop with visible tqdm
# for epoch in range(epochs):
#     model.train()
#     train_loss = 0.0
#     loop = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=True)
    
#     for batch in loop:
#         pixel_values = batch["pixel_values"].to(device)
#         labels = batch["labels"]
#         if labels.dim() == 1:
#             labels = labels.unsqueeze(0)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         with autocast(device_type="cuda", dtype=torch.float16):
#             outputs = model(pixel_values=pixel_values, labels=labels)
#             loss = outputs.loss
#             train_loss += loss.item()

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         scheduler.step()

#         loop.set_postfix(loss=loss.item())  # update progress bar with current loss

#     # clear unused GPU memory to reduce fragmentation :contentReference[oaicite:2]{index=2}
#     torch.cuda.empty_cache()

#     # --- Validation ---
#     model.eval()
#     val_loss = 0.0
#     with torch.no_grad():
#         for batch in val_dataloader:
#             pixel_values = batch["pixel_values"].to(device)
#             labels       = batch["labels"]
#             if labels.dim() == 1:
#                 labels = labels.unsqueeze(0)
#             labels = labels.to(device)

#             with autocast(device_type="cuda", dtype=torch.float16):
#                 outputs = model(pixel_values=pixel_values, labels=labels)
#                 val_loss += outputs.loss.item()

#     # logging
#     avg_train = train_loss / len(train_dataloader)
#     avg_val   = val_loss   / len(val_dataloader)
#     print(f"Epoch {epoch+1} — train_loss: {avg_train:.4f}, val_loss: {avg_val:.4f}")

#     # save checkpoint
#     model.save_pretrained(f"./training-ucf/checkpoint_{epoch+1}")
# # ───────────────────────────────────────────────────────────────────────────────

In [ ]:
# import os
# import shutil
# import torch
# import numpy as np
# import nltk
# from tqdm.auto import tqdm
# from huggingface_hub import Repository
# import evaluate

# # ─── Setup Rouge ───────────────────────────────────────────────────────────────
# nltk.download("punkt")
# rouge = evaluate.load("rouge")  # Load ROUGE metric :contentReference[oaicite:0]{index=0}

# def postprocess_text(preds, labels):
#     preds = [p.strip() for p in preds]
#     labels = [l.strip() for l in labels]
#     preds = ["\n".join(nltk.sent_tokenize(p)) for p in preds]
#     labels = ["\n".join(nltk.sent_tokenize(l)) for l in labels]
#     return preds, labels

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
#     result = rouge.compute(predictions=decoded_preds,
#                            references=decoded_labels,
#                            use_stemmer=True)
#     # scale to percentages
#     return {k: round(v * 100, 4) for k, v in result.items()}


# # ─── Prepare HF Hub Repo ────────────────────────────────────────────────────
# # Clone (or create) a HF repo into "./training-ucf"
# repo = Repository(
#     local_dir="training-ucf",
#     clone_from="your-username/ucf-captioner",
#     use_auth_token=True
# )  # uses git under the hood :contentReference[oaicite:1]{index=1}


# # ─── Training Loop ─────────────────────────────────────────────────────────────
# for epoch in range(epochs):
#     # — Training —
#     model.train()
#     train_loss = 0.0
#     pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs} • Training", leave=True)
#     for batch in pbar:
#         pixel_values = batch["pixel_values"].to(device)
#         labels = batch["labels"]
#         if labels.dim() == 1: labels = labels.unsqueeze(0)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         with autocast(device_type="cuda", dtype=torch.float16):
#             outputs = model(pixel_values=pixel_values, labels=labels)
#             loss = outputs.loss
#             train_loss += loss.item()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         scheduler.step()

#         pbar.set_postfix(loss=loss.item())

#     torch.cuda.empty_cache()

#     # — Validation & ROUGE —
#     model.eval()
#     val_loss = 0.0
#     all_preds, all_labels = [], []
#     with torch.no_grad():
#         for batch in val_dataloader:
#             pixel_values = batch["pixel_values"].to(device)
#             labels = batch["labels"]
#             if labels.dim() == 1: labels = labels.unsqueeze(0)
#             labels = labels.to(device)

#             # 1) compute loss
#             with autocast(device_type="cuda", dtype=torch.float16):
#                 outputs = model(pixel_values=pixel_values, labels=labels)
#                 val_loss += outputs.loss.item()

#             # 2) generate predictions for ROUGE
#             generated_ids = model.generate(
#                 pixel_values,
#                 max_length=labels.shape[-1],       # adjust as needed
#                 num_beams=4,                       # optional
#                 early_stopping=True
#             )
#             all_preds.extend(generated_ids.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())

#     avg_train = train_loss / len(train_dataloader)
#     avg_val   = val_loss   / len(val_dataloader)
#     rouge_scores = compute_metrics((np.array(all_preds), np.array(all_labels)))
#     print(f"Epoch {epoch+1} — train_loss: {avg_train:.4f}, "
#           f"val_loss: {avg_val:.4f}, ROUGE: {rouge_scores}")

#     # — Keep only latest checkpoint locally —
#     prev_ckpt = f"./training-ucf/checkpoint_{epoch}"
#     if epoch > 0 and os.path.isdir(prev_ckpt):
#         shutil.rmtree(prev_ckpt)  # recursively delete old dir :contentReference[oaicite:2]{index=2}

#     # — Save new checkpoint —
#     ckpt_dir = f"./training-ucf/checkpoint_{epoch+1}"
#     model.save_pretrained(ckpt_dir)
#     tokenizer.save_pretrained(ckpt_dir)

#     # — Commit & Push to Hub —
#     # Copy new checkpoint into the repo root (overwriting previous files)
#     shutil.copytree(ckpt_dir, "training-ucf", dirs_exist_ok=True)
    # repo.git_add(auto_lfs_track=True)
    # repo.git_commit(f"Epoch {epoch+1} — train_loss={avg_train:.4f}, "
    #                 f"val_loss={avg_val:.4f}, ROUGE={rouge_scores}")
    # repo.git_push()


## Editted code

# Video Captioning Training Notebook

This notebook demonstrates how to set up and train a Transformer-based video captioning model with Hugging Face. We include Markdown explanations along with code cells, and ensure proper data collation to avoid shape errors.

## 2. Dataset Definition
Inherit from `torch.utils.data.Dataset` and implement `__len__` and `__getitem__` correctly.

## 3. Load and Format Dataset
Use `load_from_disk`, set format to PyTorch tensors, and initialize our wrapper datasets.

## 4. Custom Collators
Define collators that ensure `labels` are tensors of shape `(batch_size, seq_length)` so the model's `shift_tokens_right` works without errors.

## 5. Create DataLoaders
Use our `video_collator` for both training and validation to maintain consistent batching.

## 6. Training Loop
- Use `GradScaler` with the new API.
- Accumulate gradients if needed.
- Log losses with TensorBoard.

If an OOM still occurs, consider:
- Further reducing `batch_size`.
- Enabling gradient checkpointing: `model.gradient_checkpointing_enable()`.
- Clearing cache between epochs: `torch.cuda.empty_cache()`.
- Setting `export PYTORCH_CUDA_ALLOC_CONF=split_threshold:0.5`.

## 7. Tips to Avoid OOM
- Reduce `batch_size` to 1 (or even use gradient accumulation).
- Enable gradient checkpointing: `model.gradient_checkpointing_enable()`.
- Clear GPU cache between epochs: `torch.cuda.empty_cache()`.
- Adjust PyTorch allocator config: `export PYTORCH_CUDA_ALLOC_CONF=split_threshold:0.5`.

## Save on Huggingface (push)

In [ ]:
# model_name = "training-ucf"
# hub_repo = f"NourFakih/{model_name}"

# # Save locally
# trainer.save_model(f"./{model_name}")
# tokenizer.save_pretrained(f"./{model_name}")
# feature_extractor.save_pretrained(f"./{model_name}")
# model.save_pretrained(f"./{model_name}")

# # Push to Hugging Face Hub
# tokenizer.push_to_hub(hub_repo)
# feature_extractor.push_to_hub(hub_repo)
# model.push_to_hub(hub_repo)


## Test

In [ ]:
# import os
# import av
# import numpy as np
# import pandas as pd
# import torch
# from transformers import AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel

# device = "cuda" if torch.cuda.is_available() else "cpu"
# print("Device:", device)

# # Load the models
# def load_model_and_processor(model_name):
#     processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
#     tokenizer = AutoTokenizer.from_pretrained("gpt2")
#     model = VisionEncoderDecoderModel.from_pretrained(model_name).to(device)
#     return processor, tokenizer, model

# processor1, tokenizer1, model1 = load_model_and_processor("Neleac/timesformer-gpt2-video-captioning")
# processor2, tokenizer2, model2 = load_model_and_processor("/kaggle/working/training-ucf/checkpoint_1")

# # Caption generation function
# def generate_caption(video_path, processor, tokenizer, model, num_frames=16):
#     try:
#         container = av.open(video_path)
#         total_frames = container.streams.video[0].frames
#         indices = set(np.linspace(0, total_frames, num=num_frames, endpoint=False).astype(np.int64))
#         frames = []
#         container.seek(0)
#         for i, frame in enumerate(container.decode(video=0)):
#             if i in indices:
#                 frames.append(frame.to_ndarray(format="rgb24"))
#         pixel_values = processor(frames, return_tensors="pt").pixel_values.to(device)
#         tokens = model.generate(pixel_values, min_length=10, max_length=20, num_beams=8)
#         return tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
#     except Exception as e:
#         print(f"Error processing {video_path}: {e}")
#         return "ERROR"



In [ ]:
# # Load CSV
# csv_path = "/kaggle/input/splitted-clips-ucf/splitted_clips_mapping.csv"
# video_folder = "/kaggle/input/splitted-clips-ucf/"

# df = pd.read_csv(csv_path)
# df['video_path'] = df['video_path'].str.replace(r'^\.\/splitted_clips\/', '', regex=True)
# df=df[100:105]
# # Process each row
# captions_1 = []
# captions_2 = []

# for idx, row in df.iterrows():
#     video_file = os.path.join(video_folder, row['video_path'])
#     print(f"[{idx+1}/{len(df)}] Processing {video_file}")

#     cap1 = generate_caption(video_file, processor1, tokenizer1, model1)
#     cap2 = generate_caption(video_file, processor2, tokenizer2, model2)

#     captions_1.append(cap1)
#     captions_2.append(cap2)

# # Add new columns
# df['caption_Neleac'] = captions_1
# df['caption_NourFakih'] = captions_2

# # Save to new CSV
# output_csv_path = "video_captions_augmented.csv"
# df.to_csv(output_csv_path, index=False)
# print(f"\n✅ Captions saved to {output_csv_path}")

In [ ]:
# df